## Genetski algoritem

Z genetskim algoritmom želiva ovreči spodnjo neenakost.

$ Ra(G) \geq rad(G) -1$

In [5]:
def randic(graf):
    '''Vrne randičev indeks za podan graf.'''
    vsota = 0
    povezave = graf.edges()
    stopnje = graf.degree()
    for edge in povezave:
        u, v,_ = edge
        d_u = stopnje[u]
        d_v = stopnje[v]
        vsota += 1/((d_u * d_v)**(1/2))
    return(vsota)

In [6]:
def fitness(graf):
    '''Vrne vrednost naše neenakosti. Če je vrednost negativna, lema ne drži.'''
    return randic(graf) - graf.radius() + 1

In [7]:
def fitness_populacije(populacija):
    '''Naredi seznam naborov oblike (graf, njegov fitnes).'''
    seznam = []
    for graf in populacija:
        fitnes = fitness(graf)
        seznam.append((graf, fitnes))
    return seznam

In [8]:
def tournament_selection(populacija, t):
    '''Med t naključno izbranimi grafi izbere tistega z najmanjšim fitnessom.'''
    velikost_populacije = len(populacija)
    n = randint(0, velikost_populacije - 1)
    najbolsi, fitnes_najbolsi = populacija[n]
    for i in range(1, t):
        n = randint(0, velikost_populacije - 1)
        izbrani, fitnes = populacija[n]
        if fitnes < fitnes_najbolsi:
            najbolsi = izbrani
            fitnes_najbolsi = fitnes
    return (najbolsi, fitnes_najbolsi)

S poissonom bomo izbirali število povezav, ki jih bomo v funkciji mutiraj odstranili oziroma dodali. V funkciji crossover nam pove, koliko povezav bomo dodali potomcu.

In [9]:
def poisson(t = 1, lambd = 1/2):
    '''poissonova porazdelitev'''
    N = 0
    S = 0
    while S < t:
        N += 1
        S += expovariate(lambd)
    return N

In [10]:
def mutiraj(graf):
    '''Funkcija mutira graf. Z verjetnostjo p = 1/3 doda povezavo, z p = 1/3 odstrani povezavo in z p = 1/3 doda in odstrani povezavo.'''
    kopija = Graph(graf)
    verjetnost = random()
    plus_povezave = poisson(lambd = 2) # max število povezav, ki jih bomo dodali
    minus_povezave = poisson(lambd = 2) # max število povezav, ki jih bomo odstranili
    if verjetnost <= 1/3:
        for k in range(plus_povezave):
            u, v = kopija.random_vertex(), kopija.random_vertex()
            if u != v:
                kopija.add_edge(u, v)
        return kopija
    elif verjetnost > 1/3 and verjetnost <= 2/3:
        for k in range(minus_povezave):
            povezava = kopija.random_edge()
            kopija.delete_edge(povezava)
            if not kopija.is_connected(): # če graf ni več povezan, ko odstranimo povezavo, jo moramo dodati nazaj
                kopija.add_edge(povezava)
        return kopija
    elif verjetnost > 2/3:
        for k in range(plus_povezave):
            u, v = kopija.random_vertex(), kopija.random_vertex()
            if u != v:
                kopija.add_edge(u, v)
        for k in range(minus_povezave):
            povezava = kopija.random_edge()
            kopija.delete_edge(povezava)
            if not kopija.is_connected():
                kopija.add_edge(povezava)
        return kopija

In [11]:
def crossover(a, b):
    '''Križa dva grafa med sabo in vrne njunega potomca.'''
    n = len(a)
    while True:
        podgraf_a = a.random_subgraph(0.5) # vrne podgraf grafa a, kjer je vsako vozlišče vsebovano z verjetnostjo 1/2
        podgraf_b = b.random_subgraph(0.5) # vrne podgraf grafa b, kjer je vsako vozlišče vsebovano z verjetnostjo 1/2
        if len(podgraf_a.vertices()) + len(podgraf_b.vertices()) == n and len(podgraf_a.vertices()) >= 1 and len(podgraf_a.vertices()) < n and podgraf_a.is_connected() and podgraf_b.is_connected():
            # z zgornjim pogojem želimo da sta oba podgraf povezana in da imata skupaj n vozlišč in da ni en podgraf prazen drugi pa ima n vozlišč
            podgraf_a.relabel()
            podgraf_b.relabel()
            potomec = podgraf_a.disjoint_union(podgraf_b) # povezave, ki niso skupne med grafoma
            nove_povezave = poisson(lambd = log(n/2)) # večji kot je n, več povezav bomo dodal
            for k in range(nove_povezave):
                u, v = podgraf_a.random_vertex(), podgraf_b.random_vertex()
                potomec.add_edge((0, u), (1, v))
            potomec.relabel()
            break
    return potomec

In [12]:
def zacetna_populacija(velikost, n):
    '''Naredi začetno populacijo, kjer imajo grafi n vozlišč.'''
    populacija = []
    trenutna_velikost = 0
    while trenutna_velikost < velikost:
        graf = graphs.RandomGNP(n, random()) # Naredi nek naključen graf z n vozlišči. Vsaka povezava je v grafu z neko naključno verjetnostjo.
        if graf.is_connected():
            populacija.append(graf)
            trenutna_velikost += 1
    return populacija

In [13]:
def min_fitness(seznam):
    '''V seznamu grafov poišče graf z najmanjšim fitnessom.'''
    najbolsi, najbolsi_fitnes = seznam[0]
    for nabor in seznam[1:]:
        graf, fitnes = nabor
        if fitnes < najbolsi_fitnes:
            najbolsi_fitnes = fitnes
            najbolsi = graf
    return (najbolsi, najbolsi_fitnes)

In [14]:
def nova_populacija(populacija,t):
    '''S križanjem in mutacijo grafov iz podane populacije naredi novo populacijo.'''
    nova_populacija = []
    trenutna_velikost = 0
    velikost_stare = len(populacija)
    while trenutna_velikost < velikost_stare:
        graf1 = tournament_selection(populacija, t)
        graf2 = tournament_selection(populacija, t)
        mutacija1 = mutiraj(graf1[0])
        mutacija2 = mutiraj(graf2[0])
        fit_mut1 = fitness(mutacija1)
        fit_mut2 = fitness(mutacija2)
        krizan_graf = crossover(mutacija1, mutacija2)
        fit_kriz = fitness(krizan_graf)
        minimum = min_fitness([(krizan_graf, fit_kriz), (mutacija1, fit_mut1), (mutacija2, fit_mut2), graf1, graf2])
        nova_populacija.append(minimum)
        trenutna_velikost += 1
    return nova_populacija

In [15]:
def genetic_algorithm(n, k, cas_izvajanja, t=4):
    '''Funkcija v vsaki ponovitvi naredi novo populacijo z križanjem in mutiranjem.'''
    populacija = zacetna_populacija(n, k)
    populacija = fitness_populacije(populacija)
    for i in range(cas_izvajanja):
        for nabor in populacija:
            graf, fitnes = nabor
            #print(round(fitnes, 0))
            if fitnes < 0:
                print('Lema ne drži!')
                return graf
        populacija = nova_populacija(populacija,t)
    return 'Ne najdem protiprimera.'

Časovna zahtevnost genetskega algoritma je $O$(čas_izvajanja * velikost_populacije * $n^2$).

In [16]:
genetic_algorithm(10, 12, 10, 2)

'Ne najdem protiprimera.'